<a href="https://colab.research.google.com/github/Dicarvajalb/MetNumUN2021II/blob/main/LAB11/Week4IterativeMethodsForLinearSystemsGroup8_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple iteration for systems of linear equations

First, generate a random diagonally dominant matrix, for testing.

In [ ]:
import numpy as np
np.set_printoptions(precision=1, suppress=True)
rndm = np.random.RandomState(1234)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)

print(A)

[[15.2  0.6  0.4  0.8  0.8  0.3  0.3  0.8  1.   0.9]
 [ 0.4 15.5  0.7  0.7  0.4  0.6  0.5  0.   0.8  0.9]
 [ 0.4  0.6 15.1  0.4  0.9  0.7  0.4  0.8  0.3  0.6]
 [ 0.9  0.4  0.8 15.1  0.7  0.7  0.2  0.9  0.4  0.9]
 [ 0.1  0.2  0.   0.7 15.6  0.5  0.   0.6  0.3  0.5]
 [ 0.1  0.6  0.6  0.   0.6 15.9  0.8  1.   1.   0.8]
 [ 0.3  0.6  0.5  0.2  0.4  0.1 15.5  1.   0.1  0.1]
 [ 0.7  0.6  0.5  0.1  0.2  0.9  0.4 15.5  0.   0.3]
 [ 0.4  0.6  0.9  0.6  0.7  0.1  0.7  0.8 15.6  0.4]
 [ 0.2  0.6  0.5  1.   0.5  0.5  0.5  0.8  0.1 15.7]]


# I.  Jacobi iteration

Given

$$
A x = b
$$

separate the diagonal part $D$,

$$ A = D + (A - D) $$

and write

$$
x = D^{-1} (D - A) x + D^{-1} b\;.
$$

Then iterate

$$
x_{n + 1} = B x_{n} + c\;,
$$

where 

$$
B = D^{-1} (A - D) \qquad \text{and} \qquad c = D^{-1} b
$$


Let's construct the matrix and the r.h.s. for the Jacobi iteration

In [ ]:
diag_1d = np.diag(A)

B = -A.copy() 
np.fill_diagonal(B, 0)   # A-D

D = np.diag(diag_1d)
invD = np.diag(1./diag_1d)
BB = invD @ B 
c = invD @ b

In [ ]:
# sanity checks
np.set_printoptions(precision=3, suppress=True)
from numpy.testing import assert_allclose

assert_allclose(-B + D, A)


# xx is a "ground truth" solution, compute it using a direct method
xx = np.linalg.solve(A, b)

np.testing.assert_allclose(A@xx, b)
np.testing.assert_allclose(D@xx, B@xx + b)
np.testing.assert_allclose(xx, BB@xx + c)

Check that $\| B\| \leqslant 1$:

In [ ]:
np.linalg.norm(BB)

0.36436161983015336

### Do the Jacobi iteration

In [ ]:
n_iter = 5000

x0 = np.ones(n)
x = x0
for _ in range(n_iter):
    x = BB @ x + c
print('x = ',x)

x =  [ 0.039  0.038  0.043  0.024  0.057 -0.    -0.006  0.032 -0.004  0.053]


In [ ]:
# Check the result:
np.set_printoptions(precision=4, suppress=True)
A @ x - b

array([ 0.,  0., -0., -0.,  0.,  0., -0.,  0., -0.,  0.])

### Task I.1

Collect the proof-of-concept above into a single function implementing the Jacobi iteration. This function should receive the r.h.s. matrix $A$, the l.h.s. vector `b`, and the number of iterations to perform.


The matrix $A$ in the illustration above is strongly diagonally dominant, by construction. 
What happens if the diagonal matrix elements of $A$ are made smaller? Check the convergence of the Jacobi iteration, and check the value of the norm of $B$.

(20% of the total grade)


In [ ]:
def jacobi(A, b, n_iter=1000):
    '''
    Performs a Jacobi iteration for de Ax = b system
    '''
    diag_A = np.diag(A)        # Elements of the diagonal of A. 
                               # NOTE: np.diag --> Extract a diagonal or construct a diagonal array.
        
    BB = -A.copy()             # BB is the first B matrix
    np.fill_diagonal(BB, 0)    # D-A
    D = np.diag(diag_A)        # Diagonal matrix
    invD = np.diag(1./diag_A)
    B = invD @ BB              # This is the last B matrix
    c = invD @ b
    norm = np.linalg.norm(B)
    
    x0 = np.ones(A.shape[0])
    x = x0
    for _ in range(n_iter):
        x = B @ x + c
    
    return x, norm
    

In [ ]:
jacobi(A, b)

(array([ 0.0392,  0.0378,  0.0428,  0.0237,  0.0575, -0.0003, -0.0058,
         0.0318, -0.0042,  0.0528]), 0.36436161983015336)

In [ ]:
rndm = np.random.RandomState(1234)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15] * n)    # CHANGE THIS
b = rndm.uniform(size=n)

print(A)

[[15.1915  0.6221  0.4377  0.7854  0.78    0.2726  0.2765  0.8019  0.9581
   0.8759]
 [ 0.3578 15.501   0.6835  0.7127  0.3703  0.5612  0.5031  0.0138  0.7728
   0.8826]
 [ 0.3649  0.6154 15.0754  0.3688  0.9331  0.6514  0.3972  0.7887  0.3168
   0.5681]
 [ 0.8691  0.4362  0.8021 15.1438  0.7043  0.7046  0.2188  0.9249  0.4421
   0.9093]
 [ 0.0598  0.1843  0.0474  0.6749 15.5946  0.5333  0.0433  0.5614  0.3297
   0.503 ]
 [ 0.1119  0.6072  0.5659  0.0068  0.6174 15.9121  0.7905  0.9921  0.9588
   0.792 ]
 [ 0.2853  0.6249  0.4781  0.1957  0.3823  0.0539 15.4516  0.982   0.1239
   0.1194]
 [ 0.7385  0.5873  0.4716  0.1071  0.2292  0.9     0.4168 15.5359  0.0062
   0.3006]
 [ 0.4369  0.6121  0.9182  0.6257  0.706   0.1498  0.7461  0.831  15.6337
   0.4383]
 [ 0.1526  0.5684  0.5282  0.9514  0.4804  0.5026  0.5369  0.8192  0.0571
  15.6694]]


In [ ]:
jacobi(A, b, 5000)

(array([ 0.0392,  0.0378,  0.0428,  0.0237,  0.0575, -0.0003, -0.0058,
         0.0318, -0.0042,  0.0528]), 0.36436161983015336)

You can check convergence changing the value on the diagonal. When the norm is less than 1, it converges quickly. If is close to one, you need more iterations.

# II. Seidel's iteration.

##### Task II.1

Implement the Seidel's iteration. 

Test it on a random matrix. Study the convergence of iterations, relate to the norm of the iteration matrix.

(30% of the total grade)

In [ ]:
def seidel(A, b, n_iter=1000):
    '''
    Performs a Seidel's iteration for de Ax = b system
    '''
    diag_A = np.diag(A)        # Elements of the diagonal of A. 
                               # NOTE: np.diag --> Extract a diagonal or construct a diagonal array.
    
    D = np.diag(diag_A)        # Diagonal matrix
    invD = np.diag(1./diag_A)
    U = np.triu(A) - D
    L = np.tril(A) - D
    B_tilde = -np.linalg.inv(D+L) @ U 
    c_tilde = np.linalg.inv(D+L) @ b 
    
    norm = np.linalg.norm(B)
    
    x0 = np.ones(A.shape[0])
    x = x0
    
    for _ in range(n_iter):
        x = B_tilde @ x + c_tilde
    
    return x, norm

In [ ]:
np.set_printoptions(precision=3, suppress=True)

iters = range(1,15)

for i in iters:
    print('Jacobi', jacobi(A, b, i)[0])
    print('Seidel', seidel(A, b, i)[0])
    print('')
    

Jacobi [-0.332 -0.268 -0.279 -0.36  -0.126 -0.333 -0.208 -0.204 -0.342 -0.233]
Seidel [-0.332 -0.237 -0.196 -0.185 -0.052 -0.192 -0.051  0.064  0.02   0.096]

Jacobi [0.15  0.135 0.135 0.139 0.117 0.092 0.054 0.106 0.094 0.138]
Seidel [ 0.072  0.064  0.058  0.03   0.06  -0.006 -0.011  0.028 -0.008  0.051]

Jacobi [ 0.006  0.009  0.015 -0.011  0.04  -0.028 -0.024  0.01  -0.034  0.027]
Seidel [ 0.038  0.037  0.043  0.024  0.058  0.    -0.005  0.032 -0.004  0.053]

Jacobi [ 0.049  0.046  0.051  0.034  0.063  0.008 -0.     0.038  0.005  0.061]
Seidel [ 0.039  0.038  0.043  0.024  0.057 -0.    -0.006  0.032 -0.004  0.053]

Jacobi [ 0.036  0.035  0.04   0.021  0.056 -0.003 -0.007  0.03  -0.007  0.051]
Seidel [ 0.039  0.038  0.043  0.024  0.057 -0.    -0.006  0.032 -0.004  0.053]

Jacobi [ 0.04   0.039  0.044  0.025  0.058  0.    -0.005  0.032 -0.003  0.054]
Seidel [ 0.039  0.038  0.043  0.024  0.057 -0.    -0.006  0.032 -0.004  0.053]

Jacobi [ 0.039  0.038  0.043  0.023  0.057 -0.001 -0.006

Seidel has a better convergence for the matrix A given.

# III. Minimum residual scheme

### Task III.1

Implement the $\textit{minimum residual}$ scheme: an explicit non-stationary method, where at each step you select the iteration parameter $\tau_n$ to minimize the residual $\mathbf{r}_{n+1}$ given $\mathbf{r}_n$. Test it on a random matrix, study the convergence to the solution, in terms of the norm of the residual and the deviation from the ground truth solution (which you can obtain using a direct method). Study how the iteration parameter $\tau_n$ changes as iterations progress.

(50% of the grade)

In [ ]:
def minres(A, b, n_iter=500):
    x = np.ones(b.shape[0])
    T = []
    for _ in range(n_iter):
        r = A @ x - b
        tau = (r @ A @ r)/np.linalg.norm(A @ r)**2
        x = x - tau*r
        T.append(tau)
    return x, T

x = minres(A, b)[0]

np.testing.assert_allclose(A@x, b)
np.testing.assert_allclose(x, xx)

In [ ]:
minres(A, b)[1]

[0.049458091915919704,
 0.06632815899696641,
 0.05646528371934232,
 0.05913686144833422,
 0.056149365850671325,
 0.05962056630691854,
 0.055321259622620605,
 0.060191395429619744,
 0.054726519624867076,
 0.06091835471753334,
 0.054788794256908566,
 0.06154317282841526,
 0.055177210974496134,
 0.06162125644817705,
 0.05514529510209265,
 0.06170111966684044,
 0.054725649763148955,
 0.062120810199901635,
 0.05400213360890566,
 0.06437401757338991,
 0.054433035643282154,
 0.06102904151784673,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,
 0.06413230252109958,


##Evidencias
Diego Carvajal
![](https://drive.google.com/uc?id=1Zvn9kUnVzodmZKmU09O3xtzUU52zyHnE)
Nicolas Hoyos
![](https://drive.google.com/uc?id=1FXikcX0xiVs4cWiwKvfFOzZ500LlsScK)